In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension("tabulator")

import hvplot.pandas
import holoviews as hv

In [2]:
df = pd.read_csv("survey.csv")

In [3]:
print(df.head())

            Timestamp   Age Primary streaming service  Hours per day  \
0  8/27/2022 19:29:02  18.0                   Spotify            3.0   
1  8/27/2022 19:57:31  63.0                   Pandora            1.5   
2  8/27/2022 21:28:18  18.0                   Spotify            4.0   
3  8/27/2022 21:40:40  61.0             YouTube Music            2.5   
4  8/27/2022 21:54:47  18.0                   Spotify            4.0   

  While working Instrumentalist Composer         Fav genre Exploratory  \
0           Yes             Yes      Yes             Latin         Yes   
1           Yes              No       No              Rock         Yes   
2            No              No       No  Video game music          No   
3           Yes              No      Yes              Jazz         Yes   
4           Yes              No       No               R&B         Yes   

  Foreign languages  ...  Frequency [R&B]  Frequency [Rap] Frequency [Rock]  \
0               Yes  ...        Sometimes  

## (0) Data Cleaning

In [4]:
# Drop BPM column and all NaN values
df.drop(columns=["BPM"], inplace=True)
df = df.dropna()
# DF now includes a total of 718 rows

In [5]:
# Change Age, Anxiety, Depression, Insomnia, and OCD columns to int from float
df["Age"] = df["Age"].astype("int")
df["Anxiety"] = df["Anxiety"].astype("int")
df["Depression"] = df["Depression"].astype("int")
df["Insomnia"] = df["Insomnia"].astype("int")
df1 = df["OCD"] = df["OCD"].astype("int")

In [49]:
# Mean anxiety/OCD/Depression/Insomnia rating for people who say they are instrumentalists
#table = df1.where(df1[6]=='No')  #("Instrumentalists"  #.groupby('Instrumentalist')
df_no = df[df['Instrumentalist'] == 'No'] 
df_no.head()

df_no.hvplot.bar(stacked=False, height=10, rot=60)


# mean of anxiety

KeyError: 'Instrumentalist'

In [7]:
df_yes = df[df['Instrumentalist'] == 'Yes'] 
df_yes.head()

,Timestamp,Age,Primary streaming service,Hours per day,While working,Instrumentalist,Composer,Fav genre,Exploratory,Foreign languages,...,Frequency [R&B],Frequency [Rap],Frequency [Rock],Frequency [Video game music],Anxiety,Depression,Insomnia,OCD,Music effects,Permissions
5,8/27/2022 21:56:50,18,Spotify,5.0,Yes,Yes,Yes,Jazz,Yes,Yes,...,Very frequently,Very frequently,Very frequently,Never,8,8,7,7,Improve,I understand.
6,8/27/2022 22:00:29,18,YouTube Music,3.0,Yes,Yes,No,Video game music,Yes,Yes,...,Rarely,Never,Never,Sometimes,4,8,6,0,Improve,I understand.
10,8/27/2022 22:51:15,18,Spotify,3.0,Yes,Yes,No,Country,Yes,No,...,Rarely,Never,Rarely,Never,7,7,4,7,No effect,I understand.
14,8/27/2022 23:16:06,18,Spotify,4.0,Yes,Yes,No,Jazz,Yes,Yes,...,Never,Never,Sometimes,Rarely,6,4,7,0,Improve,I understand.
24,8/28/2022 4:13:11,16,Other streaming service,3.0,Yes,Yes,Yes,Rock,Yes,Yes,...,Sometimes,Rarely,Very frequently,Sometimes,10,6,8,10,Improve,I understand.


In [8]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

## (1) Widgets (in progress)

In [9]:
# Float slider for hours per day listening to music
hours_per_day_slider = pn.widgets.FloatSlider(name="Hours per day", 
                                            start=0, 
                                            end=24, 
                                            step=0.5
                                           )
hours_per_day_slider

FloatSlider(end=24, name='Hours per day', step=0.5)

In [10]:
# Radio buttons for mental health conditions
yaxis_mental_health = pn.widgets.RadioButtonGroup(
    name="Y axis",
    options=["Anxiety", "Depression", "Insomnia", "OCD"],
    button_type="success"
)
yaxis_mental_health

RadioButtonGroup(button_type='success', name='Y axis', options=['Anxiety', 'Depression', ...], value='Anxiety')

In [11]:
# Dropdown menu for frequency by genre
select = pn.widgets.Select(name='Listening Frequency by Genre', 
                           options=['Classical', 'Country', 'EDM', "Folk", "Gospel", "Hip hop", "Jazz", "K pop", "Latin", "Lofi", "Metal", "Pop", "R&B", "Rap", "Rock", "Video game music"]
                          )
select

Select(name='Listening Frequency b..., options=['Classical', ...], value='Classical')

## (1) Mental Health Rating by Genre Preference Bar Chart

In [12]:
# Mental Health Rating by Genre Pipeline
genres = ['Classical', 'Country', 'EDM', "Folk", "Gospel", "Hip hop", "Jazz", "K pop", "Latin", "Lofi", "Metal", "Pop", "R&B", "Rap", "Rock", "Video game music"]

genre_mh_bar_pipeline = (
    idf[
        (idf["Fav genre"].isin(genres))
    ]
    .groupby("Fav genre")[yaxis_mental_health].mean()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

In [13]:
# Plotting Mental Health Rating by Genre Bar Chart
genre_mh_bar_plot = genre_mh_bar_pipeline.hvplot(kind="bar",
                                                 x="Fav genre",
                                                 y=yaxis_mental_health,
                                                 title="Mental Health Rating by Genre Preference"
                                                )
genre_mh_bar_plot

# Instrumentalist and Health Rating

In [27]:
# Radio buttons for mental health conditions
samaxis_mental_health = pn.widgets.RadioButtonGroup(
    name="Y axis",
    options=["Anxiety", "Depression", "Insomnia", "OCD"],
    button_type="success"
)

# Instrumentalist vs Health Rating Pipeline
yes_no = ["Yes", "No"]

instrumentalist_mh_bar_pipeline = (
    idf[
        (idf["Instrumentalist"].isin(yes_no))
    ]
    .groupby("Instrumentalist")[samaxis_mental_health].mean()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

AttributeError: 'DataFrameGroupBy' object has no attribute 'hvplot'

In [17]:
#Plotting Mental Health Rating by Genre Bar Chart
instrumentalist_mh_bar_plot = instrumentalist_mh_bar_pipeline.hvplot(kind="bar", stacked=True,
                                                 x="Instrumentalist",
                                                 y=samaxis_mental_health,
                                                 title="Instrumentalism vs Health Rating"
           )
instrumentalist_mh_bar_plot

# mh = ["Anxiety", "OCD", "Depression", "Insomnia"]
# instrumentalist_mh_bar_plot = instrumentalist_mh_bar_plot.hvplot(kind="bar", stacked=False)
# instrumentalist_mh_bar_plot

In [48]:
import hvplot.pandas
from bokeh.sampledata.autompg import autompg_clean as dfar
fartable = dfar.groupby('yr').mean()
fartable.hvplot.bar(y='mpg', height=500)

:Bars   [yr]   (mpg)